In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Load data

In [ ]:
from modules.data.read import DataReader

In [27]:
reader = DataReader(
    path="data/public_test/",
    output_dir="data/public_test_v6",
    tokenizer_name="sberbank-ai/rugpt3xl",
    part="test",
    window_size=40,
    local_rank=0,
    word_size=1
)

Add answer_sep: <answer>
Add start_sep <start>
Add start_sep <end>


In [28]:
reader.prc(is_save=False)

Making raw files...: 100%|██████████| 4370/4370 [00:03<00:00, 1365.57it/s]


### Generate

In [7]:
import os
os.environ["USE_DEEPSPEED"] = "1"

In [8]:
from src.xl_wrapper import RuGPT3XL

In [9]:
model = RuGPT3XL.from_pretrained(
    model_name_or_path="sberbank-ai/rugpt3xl",
    seq_len=512,
    weights_path="../models/xl/v14_130k/20000/mp_rank_00_model_states.pt",
    deepspeed_config_path="src/deepspeed_config/gpt3_xl_sparse_2048.json"
)

> initializing model parallel with size 1
Use alternating sparse & dense attention layers
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234


In [10]:
model.tokenizer = reader.tokenizer

In [11]:
import os
import torch

In [12]:
def filter_results(nr):
    return [x[:x.find("<|endoftext|>")][:x.find("</s>")] for x in nr]


def generate(model, text, additional_len=32, num_beams=5, do_sample=None):
    min_len = min(len(model.tokenizer.encode(text)), 2048 - additional_len)
    with torch.no_grad():
        return filter_results(model.generate(
            text=text,
            max_length=min_len + additional_len,
            do_sample=do_sample,
            num_beams=num_beams,
            eos_token_id=model.tokenizer.eos_token_id,
            num_return_sequences=1,
        ))[0]



In [35]:
lm_prefix = reader.lm_prefixes["named"][list(reader.lm_prefixes["named"].keys())[1]][6]

Prepare data with the following format

In [36]:
lm_prefix

'<s>человечества продемонстрировал Нюрнбергский процесс». Напомним, в пятницу и субботу в Нюрнберге пройдет международная конференция «Нюрнбергские принципы 70 лет спустя: Современные вызовы», приуроченная к 70-летию начала международного судебного процесса над нацистскими преступниками – бывшими главарями гитлеровской Германии. В работе форума, организованного<start>Международной академией нюрнбергских принципов<end>, примут участие известные юристы, историки, политики и общественные деятели из десятков стран мира. Международный трибунал, вошедший в историю как Нюрнбергский процесс, стал событием, которое явилось краеугольным камнем в конструкции современного мироустройства и легло в основу международной уголовной практики. Принципы,<answer>'

In [37]:
res = generate(model, lm_prefix, num_beams=10)

In [42]:
res.split(reader.answer_sep)[-1].strip()

'Международная академия нюрнбергских принципов'